# Installing the Keras package

In [40]:
 pip install keras

# importing  the libraries

In [41]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array


# Preprocess the Images

In [42]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [43]:
x_train = train_datagen.flow_from_directory('veg-dataset/veg-dataset/train_set', target_size = (128,128),batch_size = 32, class_mode = 'categorical')
x_test = test_datagen.flow_from_directory('veg-dataset/veg-dataset/test_set', target_size = (128,128),batch_size = 32, class_mode = 'categorical')

Found 11386 images belonging to 9 classes.
Found 3837 images belonging to 9 classes.


##  Validate the Dataset 

In [44]:
for data, labels in x_train:
    print(data.shape)
    break


(32, 128, 128, 3)


# Initialize the model

In [45]:
model = Sequential()

# Add the convolution layer

In [46]:
model.add(Convolution2D(32, (3, 3), input_shape=(128, 128, 1), activation='relu'))


# Add the pooling layer

In [47]:
model.add(MaxPooling2D(pool_size=(2,2)))

# Add the flatten layer


In [48]:
model.add(Flatten())

## Adding the dense layers 

In [49]:
model.add(Dense(units=300, kernel_initializer='uniform', activation='relu'))

model.add(Dense(units=150, kernel_initializer='uniform', activation='relu'))

model.add(Dense(units=75, kernel_initializer='uniform', activation='relu'))

model.add(Dense(units=9, kernel_initializer='uniform', activation='softmax'))


# Compile the model


In [50]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 127008)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 300)                 │      38,102,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 150)                 │          45,150 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 75)                  │          11,325 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 9)                   │             684 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,160,179 (145.57 MB)

 Trainable params: 38,160,179 (145.57 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
num_classes = model.output_shape[-1]  # This gives the number of output units (predicted classes)
print(f"Number of predicted classes: {num_classes}")


Number of predicted classes: 9


# Fit and save the model


In [53]:
model.fit(x_train, steps_per_epoch=89, epochs=20, validation_data=x_test, validation_steps=27)


Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d_3" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 128, 128, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 128, 128, 3), dtype=float32)
  • training=True
  • mask=None

In [28]:
model.save('vegetable.h5')

# load the saved model


# Test The Model


In [16]:
# Load the trained model
model = load_model("flask/vegetable.h5")

# Load the precautions DataFrame
df_precautions = pd.read_excel('precautions veg.xlsx')  # Ensure this file path is correct

# Define the function to predict and fetch precaution
def predict_precaution(img_path):
    # Load and preprocess the image
    try:
        img = image.load_img(img_path, target_size=(128, 128))  # Resize image to match model input size
        x = image.img_to_array(img)  # Convert image to array
        x = np.expand_dims(x, axis=0)  # Add batch dimension (1, 128, 128, 3)

        # Make the prediction
        pred = np.argmax(model.predict(x), axis=-1)[0]  # Get the predicted class index

        # Retrieve corresponding precaution from the DataFrame
        precaution = df_precautions.iloc[pred]['Precautions']  # Get precaution based on predicted class
        print(f"Predicted class index: {pred}")
        print(f"Precaution: {precaution}")
        
    except Exception as e:
        print(f"Error processing the image or fetching precaution: {str(e)}")

# Example usage:
img_path1 = r"C:\Users\ganin\OneDrive\Desktop\major\Veg-dataset\Veg-dataset\test_set\Pepper,_bell___Bacterial_spot\ad921dec-e88f-41d8-9455-0880c69063fc___NREC_B.Spot 9216.JPG"
img_path2 = r"C:\Users\ganin\OneDrive\Desktop\major\Veg-dataset\Veg-dataset\test_set\Pepper,_bell___healthy\b57eb9c4-59e7-4fc7-88e4-c9827304d118___JR_HL 8802.JPG"  # Replace with the path to another image

# Test with different image paths
print("Testing with image path 1:")
predict_precaution(img_path1)

print("\nTesting with image path 2:")
predict_precaution(img_path2)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'flask/vegetable.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)